# PyES samples

In [1]:
import json
from pyes import ES, Search
from pyes.aggs import TermsAgg, SumAgg, FilterAgg, DateHistogramAgg
from pyes.exceptions import IndexMissingException
from pyes.query import MatchAllQuery, BoolQuery, RangeQuery, ESRange, TermQuery
from pyes.filters import TermFilter, TermsFilter

In [2]:
message_term = TermQuery('json_data.mta_msg_id', 'abcd')

In [3]:
message_term.serialize()

{'term': {'json_data.mta_msg_id': 'abcd'}}

In [4]:
message_filter = TermFilter('json_data.mta_msg_id', 'abcd')

In [5]:
message_filter.serialize()

{'term': {'json_data.mta_msg_id': 'abcd'}}

In [6]:
import arrow
from datetime import timedelta

In [7]:
start_date = arrow.now() - timedelta(days=2)
end_date = arrow.now()

In [8]:
start_date_str = start_date.strftime('%Y-%m-%dT%H:%M:00.000Z')
end_date_str = end_date.strftime('%Y-%m-%dT%H:%M:00.000Z')

In [9]:
(start_date_str, end_date_str)

('2016-10-16T23:19:00.000Z', '2016-10-18T23:19:00.000Z')

In [10]:
term_time_range = RangeQuery(ESRange('@timestamp',
                                     from_value=start_date_str,
                                     to_value=end_date_str,
                                     include_lower=True))

In [11]:
print(json.dumps(term_time_range.serialize(), indent=2))

{
  "range": {
    "@timestamp": {
      "lt": "2016-10-18T23:19:00.000Z",
      "gte": "2016-10-16T23:19:00.000Z"
    }
  }
}


In [12]:
bool_query = BoolQuery(must=[message_filter, term_time_range])

In [106]:
print(json.dumps(bool_query.serialize(), indent=2))

{
  "bool": {
    "must": [
      {
        "term": {
          "json_data.mta_msg_id": "abcd"
        }
      },
      {
        "range": {
          "@timestamp": {
            "gte": "2016-10-16T21:13:00.000Z",
            "lt": "2016-10-18T21:13:00.000Z"
          }
        }
      }
    ]
  }
}


In [13]:
search_query = Search(bool_query,
                      bulk_read=10000,
                      _source=['json_data.*', '@timestamp'])

In [108]:
print(json.dumps(search_query.serialize(), indent=2))

{
  "_source": [
    "json_data.*",
    "@timestamp"
  ],
  "query": {
    "bool": {
      "must": [
        {
          "term": {
            "json_data.mta_msg_id": "abcd"
          }
        },
        {
          "range": {
            "@timestamp": {
              "gte": "2016-10-16T21:13:00.000Z",
              "lt": "2016-10-18T21:13:00.000Z"
            }
          }
        }
      ]
    }
  }
}


## Agg Example

In [117]:
match_all = MatchAllQuery()
sub_domain_agg = TermsAgg('domain_agg',
                          field='json_data.etp_domain_id',
                          size=20000)
client_agg = TermsAgg('client_agg',
                      field='json_data.etp_client_id',
                      sub_aggs=[sub_domain_agg],
                      size=20000)

search_query = Search(query=match_all, size=0)
search_query.agg.add(client_agg)


In [121]:
search_query.serialize()

NotImplementedError: 